# Imports de Llibreries

In [11]:
import numpy as np
import pandas as pd
import json

# Càrrega del dataset

In [31]:
with open("./data/dades_assemblea_clima.json") as file:
    file_contents = file.read()

parsed_json = json.loads(file_contents)

df = pd.json_normalize(parsed_json['components'])
df.head(5)

,id,name.translation,proposals.edges
0,3827,Propostes ciutadanes,"[{'node': {'id': '87446', 'state': 'evaluating..."


# Exploratory Data Analysis